<a href="https://colab.research.google.com/github/ludwigwittgenstein2/Research/blob/master/Novel_Address_Using_LLM_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Title: Novel Address Generation using LLM and LSTM
### by Rick Rejeleene

Generative Models, mostly we are using to generate any artifact [text, audio, video]

Feedback:

Evaluation: Include in Dissertation

How factual is it? (correct)

How accurate is it [creativity]

How measure result of generative language models?

Misused (Toxic) 

Direction: Is the generated text from? Is it from AI model or not AI model?


## First we approach using GPT-2 Model - Why?

a. GPT-2 is popular model of recent 

b. We try to use our dataset on GPT-2

c. We try to get results



###Goal of the Work: 

a. Apply Language Model 

b. Apply LSTM or variation of LSTM

c. Use Census dataset [Name-Address]

d. Compare results 

e. Write Paper/Results


In [ ]:
#install dependencies 


In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done


In [ ]:
#Check GPU 

In [ ]:
!nvidia-smi

Wed Feb  1 21:52:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 688Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.01Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 489Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 45.9Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 631Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 7.02Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.72Mit/s]


In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
#upload our own dataset 

In [ ]:
file_name = 'CleanedDatasetEntire1.txt'

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
#Train Model

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 2373.69it/s]

dataset has 17233 tokens
Training...


[10 | 26.21] loss=2.42 avg=2.42
[20 | 48.63] loss=1.95 avg=2.18
[30 | 71.71] loss=1.55 avg=1.97
[40 | 95.52] loss=1.30 avg=1.80
[50 | 119.72] loss=1.28 avg=1.69
[60 | 143.79] loss=1.02 avg=1.58
[70 | 167.87] loss=1.00 avg=1.49
[80 | 192.05] loss=0.86 avg=1.41
[90 | 216.11] loss=0.66 avg=1.32
[100 | 240.18] loss=0.61 avg=1.25
[110 | 264.29] loss=0.36 avg=1.16
[120 | 288.44] loss=0.31 avg=1.09
[130 | 312.52] loss=0.20 avg=1.02
[140 | 336.65] loss=0.12 avg=0.95
[150 | 360.79] loss=0.08 avg=0.89
[160 | 384.90] loss=0.07 avg=0.83
[170 | 409.05] loss=0.06 avg=0.78
[180 | 433.21] loss=0.05 avg=0.74
[190 | 457.38] loss=0.06 avg=0.70
[200 | 481.48] loss=0.05 avg=0.66
======== SAMPLE 1 ========
 Jr. CLARAKE ,FL 32141"
248	"Ms. Sydney Bowling , Sr, QC,"	"PO BOX 1931, IRVINE ,CA 92604"
249	"Ms. Josh Box , Sr, QA,"	"1732 SHALE ST, ALTA LOMA ,CA 91737"
250	"Ms. Victoria Box , Sr, IQCP,"	"3601 SW 131ST AVE, MARGATE ,CA 91767"
251	"Ms. Shawn Agan , Jr, QC,"	"5720 CHOLLAS BND, APT 108, WESTMINSTER ,CA 

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
import tensorflow as tf 
tf.compat.v1.reset_default_graph()

In [ ]:

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-392


In [ ]:
#Generate our data from Model

In [ ]:
"""

Consider parameters

length: Number of tokens to generate (default 1023, the maximum)
temperature: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
top_k: Limits the generated guesses to the top k guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set top_k=40)
top_p: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with top_p=0.9)
truncate: Truncates the input text until a given sequence, excluding that sequence (e.g. if truncate='<|endoftext|>', 
the returned text will include everything before the first <|endoftext|>). 
It may be useful to combine this with a smaller length if the input texts are short.
include_prefix: If using truncate and include_prefix=False, the specified prefix will not be included in the returned text.

"""

"\n\nConsider parameters\n\nlength: Number of tokens to generate (default 1023, the maximum)\ntemperature: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)\ntop_k: Limits the generated guesses to the top k guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set top_k=40)\ntop_p: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with top_p=0.9)\ntruncate: Truncates the input text until a given sequence, excluding that sequence (e.g. if truncate='<|endoftext|>', \nthe returned text will include everything before the first <|endoftext|>). \nIt may be useful to combine this with a smaller length if the input texts are short.\ninclude_prefix: If using truncate and include_prefix=False, the specified prefix will not be included in the returned text.\n\n"

In [ ]:
gpt2.generate(sess, run_name='run1')

OSM OAKS ,AR 72756"
4360 BLUE CLOVER LN, BAKERSFIELD ,CA 93312"
44	"Dr. Kara Adams , Sr, QC"	"1400 A ST, APT 6, SAN JOSE ,CA 95139"
45	"Mr. Conner Adams , Sr, QA,"	"11419 SAN LUIS ST, APT 6, CAMERON ,TX 77434"
46	"Mr. Katelyn Adams , Sr, IQCP,"	"22 PRIORY LN, ANTIOCH ,CA 94531"
47	"Mr. Mark Adams , Sr, CLIA,"	"12 SHAKERTON ST, APT 9, TUSTIN ,CA 92780"
48	"Mr. Hallie Adams , PhD, QC,"	"4514 OLD ANNETTA RD, APT A, EL MONTE ,CA 91732"
49	"Mr. Alexa Alcon , Jr, QA,"	"827 N FIG ST, APT A, MIAMI ,FL 33125"
50	"Dean. Hiskenridge Agan , Sr, IQCP,"	"1830 MARSHALL ST, HOUSTON ,TX 77044"
51	"Dr. Miles Alby , PhD, CLIA,"	"PO BOX 360, WATSONVILLE ,CA 95077"
52	"Ms. AlexagnWestLanham , PhD, QC,"	"1227 ROSEGATE BLVD, APT 8, WATSONVILLE ,CA 95077"
53	"Ms. LindsayLangham , Sr, QA,"	"366 MILL ST, STOCKTON ,CA 95207"
54	"Mr. DustyLangham , Sr, QC,"	"4630 W 6TH ST, POMONA ,CA 91766"
55	"Mr. LillianLynn , PhD, QA,"	"33519 WINSTON WAY, UNIT A, TEMECULA ,CA 92592"
56	"Col. LillianLynn , PhD, QC,"	"1060 PORT 

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Taj Mahal",
              nsamples=5,
              batch_size=5
              )

Taj Mahal , Sr, QA,"	"552 CALLE MONTEREY, SAN DIMAS ,CA 91741"
211	"Mr. Jase Mahaj , Sr, IQCP,"	"1955 MELISSA OAKS DR, GULF BREEZE ,FL 32563"
212	"Col. Aiden Miles , Sr, CLIA,"	"PO BOX 1465, ARMONA ,CA 93202"
213	"Dean. Layne Mansour , Sr, QC,"	"4521 FOREST CIR, LITTLE ROCK ,AR 72211"
214	"Dr. Drew Mehler , Jr, QA,"	"12 POINT lANCASTER ,CA 93535"
215	"Mr. Connor McNair , Jr, CLIA,"	"211 OUACHITA ROAD 17, CHIDESTER ,AR 71726"
216	"Mr. Shelby Nicks , Jr, QC,"	"107 N FELICIDAD ST, ANAHEIM ,CA 92801"
217	"Dean. Kelly Ogrick , Jr, QA,"	"304 CARMON
Taj Mahal , Sr, QA,"	"221 9TH ST, GREENFIELD ,CA 93927"
285	"Dr. Jase Mahaffy , Sr, IQCP,"	"929 TRAMELLS TRL, KISSIMMEE ,FL 34744"
286	"Col. Jessica Mahaffy , Sr, CLIA,"	"36CHINIX RD, APT LEO, DALLAS ,TX 75208"
287	"Mr. Parsa Michaelson , Sr, QC,"	"10810 MCCLEAREN DR, HOUSTON ,TX 77096"
288	"Mr. Jordan Mahaffy , Sr, QA,"	"507 5TH AVE SE, GRAVETTE ,AR 72736"
289	"Ms. Billy Munoz , Sr, IQCP,"	"895 KIETH DR, UNIT 163, OCEANSIDE ,CA 92054"
290	"Mr. An

#Text Generation using LSTM

In [ ]:
f = open('/content/word.txt', 'r')
content = f.read()


FileNotFoundError: ignored

In [ ]:

print('Corpus length:', len(text))

In [ ]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 [==============================] - 0s 0us/step
Corpus length: 600893


In [ ]:
import keras
import numpy as np

f = open('/content/CleanedDatasetEntire1.txt', 'r')
text = f.read()


In [ ]:
text

'Number\tName\tAddress\n2\t"Dean. Alex Abadi , Jr, QA, "\t"14646 RANCHERO RD, HESPERIA ,CA 92345"\n3\t"Dr. Lyla Abbatiello , Jr, CLIA, "\t"517 N MOUNTAIN AVE, # 202, UPLAND ,CA 91786"\n4\t"Mr. Kylia Abbott , Jr, QC, "\t"2731 PONKAN MEADOW DR, APOPKA ,FL 32712"\n5\t"Mrs. Bryce Abbott , Jr, QA, "\t"237 SAINT TROPEZ LN, LINCOLN ,CA 95648"\n6\t"Ms. Daniela Abella , PhD, IQCP,"\t"20054 CHICKEN DR, DECATUR ,AR 72722"\n7\t"Mr. Jacob Abraham , Jr, CLIA, "\t"3006 E 17TH ST, # 6, OAKLAND ,CA 94601"\n8\t"Mr. Reily Acevedo , PhD, QC,"\t"15123 BROOKHURST ST, APT 357, WESTMINSTER ,CA 92683"\n9\t"Col. Micaela Acevedo , PhD, QA, "\t"1724 HORIZON HEIGHTS CIR, EL CAJON ,CA 92019"\n10\t"Mr. Breonia Adam , PhD, QC, "\t"513 LANDWYCK LN, FLOWER MOUND ,TX 75028"\n11\t"Mr. Miranda Adam , Jr, QA,"\t"3503 E CORNELL ST, LUBBOCK ,TX 79403"\n12\t"Mr. Madelyn Adams , Sr, CLIA"\t"102 HILLVIEW DR, CLOVERDALE ,CA 95425"\n13\t"Mr. Jase Adams , PhD, QC, "\t"1940 MARKET ST, FERNDALE ,CA 95536"\n14\t"Col. Jessica Adams , 

In [ ]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print(chars)

print('Unique characters:', len(chars))

# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 12749
['\t', '\n', ' ', '"', '#', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '¬', 'É', '√']
Unique characters: 75
Vectorization...


<ipython-input-59-225bf8033284>:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-59-225bf8033284>:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
from keras import layers
import tensorflow as tf

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(tf.keras.layers.Dense(4))
model.add(layers.Dense(len(chars), activation='softmax'))

#model.add(layers.Dense(len(chars), activation='relu'))


In [ ]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_25 (LSTM)              (None, 128)               104448    
                                                                 
 dense_17 (Dense)            (None, 75)                9675      
                                                                 
Total params: 114,123
Trainable params: 114,123
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
    #for temperature in [1.2, 1.5, 2.0, 2.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Train on 12749 samples
12749/12749 [==============================] - 4s 325us/sample - loss: 1.4909
--- Generating with seed: "108	"Ms. Kira Arney , Jr,"	"CLIA, 11320 BAIRD AVE, PORTER RA"
------ temperature: 0.2
108	"Ms. Kira Arney , Jr,"	"CLIA, 11320 BAIRD AVE, PORTER RANTEN ,CA 92008"
371	"Mr. Kenn Bernnn , Jr, QA,"	"111111115 CHIALE AVE, SAN ANTEN ,CA 91700"
171	"Mr. Allen Bernnn , Jr, QA,"	"11170 SAN ANTEN DR, LONTEN ,CA 91720"
241	"Mr. Kennnn Bernnn , Jr, QC,"	"10111 W STONT MAN ANTEN AVE, LAN ANTEN ,CA 91707"
272	"Mr. Annen Bernnn , Jr, QA,"	"1111111311115	"Mr. Jarnnnn Bernnn , Jr, QA,"	"17111 W 17TH AVE, TONTEN ,CA 91707"
378	"Mr. Jannnnn Bernnn , Jr, QA,"	
------ temperature: 0.5
H AVE, TONTEN ,CA 91707"
378	"Mr. Jannnnn Bernnn , Jr, QA,"	"1017 FANDLE DR, FON DENTE ,CA 91760"
391	"Mr. Kiman Bernnn , Jr, QA,"	"111112 DALELO DR, LONT ANTEN ,CA 91761"
127	"Dean . Mennn Beant , Jr, QA,"	"1003 ACHELLE CT, FENTIN ,TX 78376"
16	"Mr. Jyennnn Bronnn , Jr, QA,"	"14110 MANDE CTO

KeyboardInterrupt: ignored

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize

def calculate_bleu(candidate, reference):
    '''
    candidate, reference: generated and ground-truth sentences
    '''
    reference = word_tokenize(reference)
    candidate = word_tokenize(candidate)
    score = sentence_bleu(reference, candidate)
    return score

In [ ]:
def measure_novelty(text):
    # split the text into words
    words = text.split()

    # get the number of unique words
    num_unique_words = len(set(words))

    # return the number of unique words as a measure of novelty
    return num_unique_words

# example text
text = "Let me not to the marriage of true minds Let me not to the marriage of true minds Admit impediments; love is not love Which alters when it alteration finds"

# measure the novelty of the text
novelty = measure_novelty(text)

# print the novelty score
print("Novelty:", novelty)

In [ ]:
!pip install math

In [ ]:
import math

def measure_novelty(text, model):
    # get the probability of the text under the model's distribution
    p = model.probability(text)

    # compute the novelty score using the f(text) = -log p(text) function
    score = -math.log(p)

    # return the novelty score
    return score

# example text and model
text = "This is an example of some text that we want to measure for novelty"
model = ...

# measure the novelty of the text
novelty = measure_novelty(text, model)

# print the novelty score
print("Novelty:", novelty)

In [ ]:
# import the necessary library
import numpy as np

# define a custom model class
class Model:
    def __init__(self):
        # initialize any necessary model parameters here
        pass

    def probability(self, word, prev_word):
        # compute and return the probability of the given word given the previous word
        pass

def compute_probability(text, model):
    # split the text into words
    words = text.split()

    # initialize the probability to 1
    p = 1

    # loop over the words in the text
    for i in range(len(words)):
        # get the current word and the previous word
        word = words[i]
        prev_word = words[i-1] if i > 0 else None

        # compute the probability of the current word given the previous word
        word_prob = model.probability(word, prev_word)

        # multiply the probability by the probability of the current word
        p *= word_prob

    # return the probability of the text
    return p

# create an instance of the custom model class
model = Model()

# example text
text = "This is an example of some text that we want to compute the probability of"

# compute the probability of the text under the model's distribution
p = compute_probability(text, model)

# print the probability
print("Probability:", p)


In [ ]:
def compute_probability(text, model):
    # split the text into words
    words = text.split()

    # initialize the probability to 1
    p = 1

    # loop over the words in the text
    for i in range(len(words)):
        # get the current word and the previous word
        word = words[i]
        prev_word = words[i-1] if i > 0 else None

        # compute the probability of the current word given the previous word
        word_prob = model.probability(word, prev_word)

        # check if the probability is None
        if word_prob is None:
            # if the probability is None, return 0
            return 0

        # multiply the probability by the probability of the current word
        p *= word_prob

    # return the probability of the text
    return p


In [ ]:
def compute_probability(text, model):
    # split the text into words
    words = text.split()

    # print the words
    print("Words:", words)

    # initialize the probability to 1
    p = 1

    # loop over the words in the text
    for i in range(len(words)):
        # get the current word and the previous word
        word = words[i]
        prev_word = words[i-1] if i > 0 else None

        # compute the probability of the current word given the previous word
        word_prob = model.probability(word, prev_word)

        # print the current word and its probability
        print("Word:", word)
        print("Probability:", word_prob)

        # check if the probability is None
        if word_prob is None:
            # if the probability is None, return 0
            return 0

        # multiply the probability by the probability of the current word
        p *= word_prob

    # return the probability of the text
    return p


In [ ]:
import torch
import transformers as tfm

def detect_novelty(text, model):
    # split the text into words
    words = text.split()

    # initialize the novelty score to 0
    novelty_score = 0

    # loop over the words in the text
    for i in range(len(words)):
        # get the current word and the previous word
        word = words[i]
        prev_word = words[i-1] if i > 0 else None

        # compute the probability of the current word given the previous word
        # using the transformer model
        word_tokens = model.tokenizer.encode(word, prev_word)
        word_prob = model.model(torch.tensor([word_tokens]))[0][-1][-1]

        # add the log probability of the current word to the novelty score
        novelty_score += np.log(word_prob)

    # return the novelty score
    return novelty_score

# example text and transformer model
text = "This is an example of some text that we want to compute the novelty score of"
model = tfm.BertModel.from_pretrained('bert-base-uncased')

# compute the novelty score of the text
novelty_score = detect_novelty(text, model)

# print the novelty score
print("Novelty score:", novelty_score)


In [ ]:
import nltk
nltk.download('punkt')

def score_novelty(paragraph, text):
  # Tokenize the paragraph and text using NLTK
  tokens = nltk.word_tokenize(paragraph)
  text_tokens = nltk.word_tokenize(text)

  # Create a set of unique tokens from the paragraph
  unique_tokens = set(tokens)

  # Count the number of unique tokens in the text
  num_unique_tokens = len(set(text_tokens).intersection(unique_tokens))

  # Calculate the novelty score by dividing the number of unique tokens in the text by the total number of tokens in the text
  novelty_score = num_unique_tokens / len(text_tokens)

  return novelty_score

# Example usage
paragraph = "Many factories were evacuated, together with much of the government, and from October 20 the city was declared to be in a state of siege. Its remaining inhabitants built and manned antitank defenses, while the city was bombarded from the air. On May 1, 1944, a medal For the defence of Moscow and in 1947 another medal In memory of the 800th anniversary of Moscow was instituted."
text = "Many factories were evacuated,"
novelty_score = score_novelty(paragraph, text)
print(novelty_score)  # Output: 0.4


In [ ]:
import csv
import math
from collections import Counter

def cosine_similarity(text1, text2):
    # Convert the texts to a Counter object that stores the frequency of each word
    counter1 = Counter(text1)
    counter2 = Counter(text2)
    
    # Find the intersection of the words in both texts
    words = set(counter1.keys()).intersection(set(counter2.keys()))
    
    # Calculate the dot product of the frequency vectors of the common words
    dot_product = sum(counter1[word] * counter2[word] for word in words)
    
    # Calculate the magnitude of each frequency vector
    magnitude1 = math.sqrt(sum(counter1[word] ** 2 for word in counter1.keys()))
    magnitude2 = math.sqrt(sum(counter2[word] ** 2 for word in counter2.keys()))
    
    # Calculate and return the cosine similarity score
    return dot_product / (magnitude1 * magnitude2)

def read_text_from_csv(file_name):
    text = []
    with open(file_name, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            text.extend(row)
    return text

# Read the text from the CSV files
text1 = read_text_from_csv("text1.txt")
text2 = read_text_from_csv("text3.csv")
print(text1)
print(text2)

# Calculate the cosine similarity of the two texts
similarity = cosine_similarity(text1, text2)

print("Cosine Similarity:", similarity)


In [ ]:
# import required libraries
import numpy as np
from numpy.linalg import norm

# define two lists or array
A = np.array([2,1,2,3,2,9])
B = np.array([3,4,2,4,5,5])

print("A:", A)
print("B:", B)

# compute cosine similarity
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)
